## Dependencies

In [1]:
!pip install --upgrade google-api-python-client
!pip install bitsandbytes>=0.39.0
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install tiktoken
!pip install torch
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 47.9 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 8.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-uaf34nd3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-uaf34nd3
  Resolved https://github.com/huggingface/transformers.git to commit 30ed3adf474aaf2972ab56f5624089bc24a6adf3
  Installing bu

In [2]:
!nvidia-smi

Mon Jul 10 12:38:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  Off  | 00000000:00:0B.0 Off |                    0 |
| N/A   40C    P0    25W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
!free

              total        used        free      shared  buff/cache   available
Mem:      181108932    10473132    73337164     5609268    97298636   163382608
Swap:             0           0           0


## Imports

In [13]:
import os
import torch
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## Loading the model

In [14]:
model_name = 'legendhasit/xgen-7b-8k-inst-8bit'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


## Inference

In [4]:
# General settings for zero-shot sampling
header = (
    "A chat between a curious human and an artificial intelligence assistant. "
    "The assistant gives helpful, detailed, and polite answers to the human's questions.\n"
)

article = """
E2: Dis voir Christiane, tu vas descendre bientôt chez ta soeur à Istres?
C: (bruit de papier) Ah ben euh, on descend pour le mariage de Myriam et, et Emmanuel.
C: Oui, (XX) sont en plein préparatifs, bien entendu.
C: La petite Arlésienne, elle est mignonne comme tout, enfin bon.
E2: Et alors elle aura la robe de l'Arlésienne non?
C: Oh non, je pense pas qu'elle ait un costume régional, je pense.
C: Je pense qu'elle aura une tenue euh <E2: De mariage traditionnel.> classique de mariée. Je pense.
C: Remarque, on sait jamais hein, ça peut être la surprise, j'aimerais bien parce que c'est,
C: c'est vrai que c'est joli, le costume d'Arlésienne. <Y: C'est très folklorique, oui, oui.>
C: Je sais pas si elle, <Y: C'est très, très joli.> si elle en a un, (bruit de papier) j'ai jamais eu l'occasion de, <Y: Mais c'est fort possible que pour le mariage elle ait euh.> de voir.
"""

In [5]:

with open('prompts.txt', 'r', encoding='utf-8') as input_file:
    lines = input_file.readlines()
    counter = 0
    for line in lines:

        # Read instruction and create prompt
        instruction = line.replace('\n', '')
        prompt = header + f"\n### Human: {instruction}\n{article}\n###"

        # Generate answer
        input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
        sample = model.generate(**input_ids, do_sample=True, max_new_tokens=2048, top_k=100, eos_token_id=50256)
        output = tokenizer.decode(sample[0]).strip()
        
        # Save answer in file
        os.mkdir('output')
        with open('output/' + counter + '.txt', encoding='utf-8') as target_file:
            target_file.write(output)
            target_file.close()


A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human's questions.

### Human: Résume le texte suivant en un paragraphe monobloc.

E2: Dis voir Christiane, tu vas descendre bientôt chez ta soeur à Istres?
C: (bruit de papier) Ah ben euh, on descend pour le mariage de Myriam et, et Emmanuel.
C: Oui, (XX) sont en plein préparatifs, bien entendu.
C: La petite Arlésienne, elle est mignonne comme tout, enfin bon.
E2: Et alors elle aura la robe de l'Arlésienne non?
C: Oh non, je pense pas qu'elle ait un costume régional, je pense.
C: Je pense qu'elle aura une tenue euh <E2: De mariage traditionnel.> classique de mariée. Je pense.
C: Remarque, on sait jamais hein, ça peut être la surprise, j'aimerais bien parce que c'est,
C: c'est vrai que c'est joli, le costume d'Arlésienne. <Y: C'est très folklorique, oui, oui.>
C: Je sais pas si elle, <Y: C'est très, très joli.> si elle en a un, (bruit de papier) j'ai

In [10]:
del input_ids
del sample
del output

NameError: name 'sample' is not defined